In [1557]:
import pandas as pd
import numpy as np

In [1558]:
parts_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/parts_data.csv')
#parts_data.drop(columns=['Quantity Received YTD'], inplace=True)

parts_data.drop_duplicates(subset=['Part Number'], inplace=True)
parts_data = parts_data.drop(columns=['Unnamed: 0'])

In [1559]:
parts_data['Cost Per Unit'] = parts_data['Price'] - parts_data['Margin']
parts_data['Total Cost'] = parts_data['Cost Per Unit'] * abs(parts_data['Quantity'])

In [1560]:
def fix_text(data_frame, column_name):
    if (column_name != 'Part Number') & (column_name != 'Category'):
        data_frame[column_name] = data_frame[column_name].str.lower().str.strip()
    else:
        data_frame[column_name] = data_frame[column_name].str.strip()
    
    return data_frame

text_cols = parts_data.select_dtypes(exclude='number').columns
for col in text_cols:
    parts_data = fix_text(parts_data, col)

In [1561]:
parts_data.head()

,Part Number,Category,Description,Supplier Name,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,...,Sales - 1st Qtr,Sales - 2nd Qtr,Sales - 3rd Qtr,Sales - 4th Qtr,Sales - This Year,Sales - Last Year,Quantity Ordered YTD,Quantity Received YTD,Cost Per Unit,Total Cost
0,09148-04032,SHP,nut,**added from pricebook**,25.95,0.89,3.43,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0,2.54,0.0
1,09320-04501,SHP,cushion,**added from pricebook**,25.07,0.92,3.67,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0,2.75,0.0
2,09329-10026,SHP,cushion,**added from pricebook**,37.37,1.11,2.97,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0,1.86,0.0
3,10-08207-51,ACC,painted dsr jet black with,**added from pricebook**,15.99,463.00,2895.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0,2432.00,0.0
4,25-08031,ACC,fx/mx brake lever jjuan,**added from pricebook**,35.01,17.50,49.99,0.0,0.0,0.0,...,0.0,1.0,0.0,0,1.0,0.0,0,43,32.49,0.0


In [1562]:
parts_data.dropna(subset=['Part Number'], inplace=True)
parts_data.dropna(subset=['Price'], inplace=True)

In [1563]:
def drop_negatives(data_frame, column_name, threshold=0):
    mask = data_frame[column_name] <= threshold
    if mask.any():
        data = data_frame[~mask]
        return data
    else:
        return data_frame

# Example usage
parts_data = drop_negatives(parts_data, 'Price')


In [1564]:
parts_data['Obsolete'] = parts_data['Months No Sale'].apply(lambda x: 1 if x >= 12 else 0)

In [1565]:
#convert things to main suppliers and other suppliers just to simplify, then one hot encode everything
supplier_cost = parts_data.groupby('Supplier Name')['Total Cost'].sum().sort_values(ascending=False)
round(supplier_cost, 2)

Supplier Name
polaris                         354064.82
triumph                         315604.77
bmw                             259881.17
ducati                          182132.79
kimpex                           69573.59
                                  ...    
fastener force one                   0.00
first gear                           0.00
fortnine                             0.00
fred kolman's wheelsport ltd         0.00
ztechnik                             0.00
Name: Total Cost, Length: 237, dtype: float64

In [1566]:
historical_cols = ['Sales Last Month',
       'Sales Last 3 Months', 'Sales Last 6 Months',
       'Sales Last 9 Months', 'Sales Last 12 Months', 
       'Sales Last 2 Years','Sales Last 3 Years', 
       'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years']

this_year_cols = ['Sales - Jan', 'Sales - Feb', 'Sales - Mar',
       'Sales - Apr', 'Sales - May', 'Sales - Jun', 'Sales - Jul',
       'Sales - Aug', 'Sales - Sep', 'Sales - Oct', 'Sales - Nov', 'Sales - Dec']


In [1567]:
def calculate_metric(parts_data, time_frame_cols, metric, column_to_multiply, total_sales_year):
    for col in time_frame_cols:
        # Extract the time frame (e.g., 'Jan', 'Feb', etc.) from the column name
        time_frame = col.split('-')[-1].strip()

        # Create a new column name for the metric (e.g., 'Gross Profit - Jan', etc.)
        metric_col = f'{metric} - {time_frame}'

        # Calculate the metric for the current time frame
        parts_data[metric_col] = parts_data[column_to_multiply] * parts_data[col]

    parts_data[f'Total {metric}'] = parts_data[total_sales_year] * parts_data[column_to_multiply]

    return parts_data

# Calculate Revenue for historical time frames (assuming you have a column 'Cost' representing the cost of each part)
parts_data = calculate_metric(parts_data, historical_cols, 'Gross Profit', 'Price', 'Sales - Last Year')

# Calculate Profits for this year time frames (assuming you have a column 'Cost' representing the cost of each part)
parts_data = calculate_metric(parts_data, this_year_cols, 'Gross Profit', 'Price', 'Sales - This Year')

In [1568]:
cols_to_rename = ['Gross Profit - Sales Last Month', 
       'Gross Profit - Sales Last 3 Months',
       'Gross Profit - Sales Last 6 Months',
       'Gross Profit - Sales Last 9 Months',
       'Gross Profit - Sales Last 12 Months',
       'Gross Profit - Sales Last 2 Years',
       'Gross Profit - Sales Last 3 Years',
       'Gross Profit - Sales Last 4 Years',
       'Gross Profit - Sales Last 5 Years',
       'Gross Profit - Sales Last 10 Years']

# Define a mapping of current column names to new column names
column_mapping = {col: col.replace('Sales', '') for col in cols_to_rename}

# Rename the columns using the .rename() method
parts_data.rename(columns=column_mapping, inplace=True)


In [1569]:
def negative_sales(data_frame, column_names):
    """
    The reason there are negative sales numbers is to reflect inventory that was sold prior to being added correctly into the dealership ERP,
    this is common practice during busier months as volume dramatically increases and inventory can only be added to the system every so often.
    Additionally, inventory counts occur 1 - 2 times a year which is why many of the issues such as this persist across time

    Usage: provide dataframe and columns to switch and will set negative columns to their absolute value
    """

    data_frame = data_frame.copy()  # Make a copy to avoid the warning

    for col in column_names:
        if np.issubdtype(data_frame[col].dtype, np.number):  # Only apply to numeric columns
            data_frame.loc[data_frame[col] < 0, col] = abs(data_frame[col])

    return data_frame

# Usage:
column_names = ['Sales - Last Year', 
    'Sales - Jan', 'Sales - Feb', 'Sales - Mar',
    'Sales - Apr', 'Sales - May', 'Sales - Jun', 'Sales - Jul',
    'Sales - Aug', 'Sales - Sep', 'Sales - Oct', 'Sales - Nov', 'Sales - Dec',
    'Sales - This Year', 
    'Sales Last Month', 'Sales Last 3 Months', 'Sales Last 6 Months',
    'Sales Last 9 Months', 'Sales Last 12 Months', 'Sales Last 2 Years',
    'Sales Last 3 Years', 'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years']

parts_data = negative_sales(parts_data, column_names)

In [1570]:
parts_data.describe()

,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,Sales Last 6 Months,Sales Last 9 Months,Sales Last 12 Months,Sales Last 2 Years,...,Gross Profit - Mar,Gross Profit - Apr,Gross Profit - May,Gross Profit - Jun,Gross Profit - Jul,Gross Profit - Aug,Gross Profit - Sep,Gross Profit - Oct,Gross Profit - Nov,Gross Profit - Dec
count,89220.000000,89220.000000,89220.000000,89220.000000,89220.000000,89220.000000,89220.000000,89220.000000,89220.000000,89220.000000,...,89220.000000,89220.000000,89220.000000,89220.000000,89220.000000,89220.000000,89220.0,89220.0,89220.0,89220.0
mean,39.882482,84.586276,231.098894,0.529002,0.037974,0.105890,0.179466,0.288916,0.338712,0.532947,...,2.799296,2.716837,3.143837,3.559755,2.838842,0.237714,0.0,0.0,0.0,0.0
std,332.826987,215.354620,587.793020,5.085519,1.185032,2.894954,4.705985,6.541851,7.652564,12.005605,...,50.531913,50.307208,51.948548,54.488264,60.710795,10.435994,0.0,0.0,0.0,0.0
min,-98600.000000,-3305.000000,0.010000,-12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-540.990000,-1256.990000,-1559.220000,-1919.040000,-579.990000,-389.990000,0.0,0.0,0.0,0.0
25%,33.230000,8.080000,21.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,38.345000,28.665000,75.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,48.610000,90.690000,241.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,100.000000,15299.000000,29060.990000,554.000000,216.000000,421.000000,695.000000,850.000000,1137.000000,2196.000000,...,5893.990000,4000.990000,4318.990000,4197.960000,10250.990000,1009.990000,0.0,0.0,0.0,0.0


In [1571]:
def update_column_with_condition(dataframe, target_column, condition_columns, replacement_column):
    mask = (dataframe[condition_columns[0]] > 0) | (dataframe[condition_columns[1]] > 0) | (dataframe[condition_columns[2]] > 0)
    dataframe[target_column] = np.where(mask, replacement_column, dataframe[target_column])
    return dataframe

cols = ['Part Number', 'Description', 'Supplier Name', 'Price', 'Quantity', 'Months No Sale', 'Sales Last Month', 'Sales Last 3 Months', 
        'Sales Last 6 Months', 'Sales Last 9 Months', 'Sales Last 12 Months', 'Sales - This Year', 'Turnover', 'Total Cost', 'COGS']

parts_data = update_column_with_condition(parts_data, 'Sales - This Year',
                                                   ['Sales Last Month', 'Sales Last 3 Months', 'Sales Last 6 Months'],
                                                   parts_data['Sales Last 6 Months'])

# Calculate the target for updating 'Total Cost'
target = parts_data['Cost Per Unit'] * parts_data['Quantity'].abs()

# Update 'Total Cost' based on the target calculation
parts_data = update_column_with_condition(parts_data, 'Total Cost',
                                                   ['Sales Last Month', 'Sales Last 3 Months', 'Sales Last 6 Months'],
                                                   target)

In [1572]:
mask = parts_data['Total Cost'] == 0.00

# Update 'Margin' column based on the condition
parts_data['Margin'] = np.where(mask, parts_data['Price'], parts_data['Margin'])

parts_data['Margin %'] = (parts_data['Margin'] / parts_data['Price']) * 100


In [1573]:
mask = ((complete_parts_data['Months No Sale'] <= 8) & (complete_parts_data['Sales Last 9 Months'] > 0) & complete_parts_data['Sales - This Year'] == 0)

complete_parts_data['Sales - This Year'] = np.where(mask, complete_parts_data['Sales Last 9 Months'], complete_parts_data['Sales - This Year'])

In [1574]:
cost_mask = ((complete_parts_data['Turnover'] == 0) & (complete_parts_data['Months No Sale'] <= 8) & (complete_parts_data['Total Cost'] == 0))

cols = ['Part Number', 'Description', 
                    'Supplier Name', 'Price', 'Quantity', 'Months No Sale', 'Sales Last Month', 'Sales Last 3 Months', 
                    'Sales Last 6 Months', 'Sales Last 9 Months', 'Sales Last 12 Months', 'Sales - This Year', 'Turnover', 'COGS', 'Total Cost']

complete_parts_data[cost_mask][cols]

,Part Number,Description,Supplier Name,Price,Quantity,Months No Sale,Sales Last Month,Sales Last 3 Months,Sales Last 6 Months,Sales Last 9 Months,Sales Last 12 Months,Sales - This Year,Turnover,COGS,Total Cost
0,0,return spring,triumph,7.26,0,0,0,0,0,0,0,0,0.0,0.0,0.0
1,0 580 463 999,euro moto internationalrelectrics bosch,euro moto internationalrelectrics,629.99,0,0,0,0,0,0,0,0,0.0,0.0,0.0
2,00-0450,"screw, gearbox,inspec&",moto international,1.95,0,0,0,0,0,0,0,0,0.0,0.0,0.0
4,00-F01740401,turn signal light control,moto international,166.99,0,0,0,0,0,0,0,0,0.0,0.0,0.0
5,00-F02600371,1399 belt drive,moto international,51.99,0,0,0,0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89198,mv,did tip for km500 cutter,moto international,15.99,0,0,0,0,0,0,0,0,0.0,0.0,0.0
89202,pazzo racing inc. MISC,misc pazzo racing inc. part,pazzo racing inc.,104.99,0,0,0,0,0,0,0,0,0.0,0.0,0.0
89208,s-bm0112,r1100gs 93-99 front/rear 5 lines abs2,spiegler,289.99,0,8,0,0,0,1,1,1,0.0,0.0,0.0
89216,t2501733,ytx12-bs,savage cycles,100.99,0,8,0,0,0,1,1,1,0.0,0.0,0.0


In [1575]:
def fix_sales(dataframe, sales_periods):
    for i, sale in enumerate(sales_periods):
        if (dataframe[sale][i] > 0) & (dataframe[sale][1 + 1] == 0):
            dataframe[sale + 1] += dataframe[sale]
        else:
            dataframe[sale]
    return dataframe


sales_figures = ['Sales Last Month',
       'Sales Last 3 Months', 'Sales Last 6 Months', 'Sales Last 9 Months',
       'Sales Last 12 Months', 'Sales Last 2 Years', 'Sales Last 3 Years',
       'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years']

fix_sales(complete_parts_data, sales_figures)

,Part Number,Category,Description,Supplier Name,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,...,Gross Profit - Aug,Gross Profit - Sep,Gross Profit - Oct,Gross Profit - Nov,Gross Profit - Dec,COGS,Starting Inventory,Ending Inventory,Average Inventory,Turnover
0,0,PM1,return spring,triumph,100.0,7.26,7.26,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0 580 463 999,ACC,euro moto internationalrelectrics bosch,euro moto internationalrelectrics,100.0,629.99,629.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00-0450,ACC,"screw, gearbox,inspec&",moto international,100.0,1.95,1.95,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00-548588,ATA,tube 150/90- 15 centre90' stem,butler maps inc,100.0,19.99,19.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00-F01740401,ACC,turn signal light control,moto international,100.0,166.99,166.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89215,t2048903,PGC,"yoke assy, lower",savage cycles,100.0,392.99,392.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89216,t2501733,ABT,ytx12-bs,savage cycles,100.0,100.99,100.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89217,t3010108,ACC,flanged sleeve m6,triumph,100.0,2.89,2.89,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89218,t3330696,PGC,screw,savage cycles,100.0,5.99,5.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1576]:
# Calculate the required metrics in the parts_data DataFrame
total_cogs = round((parts_data['Cost Per Unit'] * parts_data['Sales - This Year']).sum(), 2)
total_end_inventory = round((parts_data['Price'] * parts_data['Quantity']).sum(), 2)
total_start_inventory = (total_end_inventory - parts_data['Quantity Ordered YTD'].sum()) + total_cogs
avg_total_inventory = round((total_start_inventory + total_end_inventory) / 2, 2)
total_turnover = total_cogs / avg_total_inventory if avg_total_inventory > 0 else 0

# Create a list of dictionaries for the tabular chart
data = [
    {
        "Metric": "Start Inventory",
        "Value": total_start_inventory
    },
    {
        "Metric": "End Inventory",
        "Value": total_end_inventory
    },
    {
        "Metric": "Average Inventory",
        "Value": avg_total_inventory
    },
    {
        "Metric": "COGS",
        "Value": total_cogs
    },
    {
        "Metric": "Turnover",
        "Value": total_turnover
    }
]

# Print the tabular chart
table = pd.DataFrame(data, columns=['Metric', 'Value'])
table

,Metric,Value
0,Start Inventory,3.337462e+06
1,End Inventory,2.269005e+06
2,Average Inventory,2.803234e+06
3,COGS,1.073196e+06
4,Turnover,3.828423e-01


In [1577]:
def create_supplier_dataframe(dataframe, suppliers=None):
    """
    Create a data frame with the turnover, cogs, and average inventory for suppliers
    
    Usage: Takes 1 or more suppliers and returns a DataFrame with the corresponding values.
           The default value if no supplier is given is all the suppliers in the data set
    """
    if suppliers is None:
        suppliers = dataframe['Supplier Name'].unique()
    
    # Filter data based on suppliers
    suppliers_data = dataframe[dataframe['Supplier Name'].isin(suppliers)]
    
    # Calculate COGS, end inventory, start inventory, average inventory, and turnover
    suppliers_data['COGS'] = suppliers_data['Cost Per Unit'] * suppliers_data['Sales - This Year']
    suppliers_data['Ending Inventory'] = suppliers_data['Price'] * suppliers_data['Quantity'].abs()
    suppliers_data['Starting Inventory'] = (suppliers_data['Ending Inventory'] - suppliers_data['Quantity Ordered YTD']) + suppliers_data['COGS']
    suppliers_data['Average Inventory'] = (suppliers_data['Starting Inventory'] + suppliers_data['Ending Inventory']) / 2
    suppliers_data['Turnover'] = np.where(suppliers_data['Average Inventory'] <= 0, 0, suppliers_data['COGS'] / suppliers_data['Average Inventory'])
    suppliers_data.loc[suppliers_data['COGS'] == 0, 'Turnover'] = suppliers_data['Sales - This Year']

    # Group by supplier and aggregate the data
    result_df = suppliers_data.groupby(dataframe['Supplier Name']).agg({
        'Quantity': 'sum',
        'Months No Sale': 'mean',
        'Sales Last Month': 'mean',
        'Sales Last 3 Months': 'mean',
        'Sales Last 6 Months': 'mean',
        'Sales Last 9 Months': 'mean',
        'Sales Last 12 Months': 'mean',
        'Sales - This Year': 'mean',
        'Total Cost': 'mean',
        'COGS': 'sum',
        'Starting Inventory': 'sum',
        'Ending Inventory': 'sum',
        'Average Inventory': 'sum',
        'Turnover': 'mean'
    }).reset_index()

    return result_df
  
supplier_df = create_supplier_dataframe(parts_data)

In [1578]:
def create_part_dataframe(dataframe, part_numbers=None):
    if part_numbers is None:
        part_numbers = dataframe['Part Number'].unique()
    
    part_data = dataframe[dataframe['Part Number'].isin(part_numbers)].copy()
    
    part_data['COGS'] = part_data['Cost Per Unit'] * part_data['Sales - This Year']

    part_data['Ending Inventory'] = part_data['Price'] * part_data['Quantity'].abs()

    part_data['Starting Inventory'] = np.where((part_data['Quantity Received YTD'] <= part_data['Quantity Ordered YTD']).any(),
                                                (part_data['Ending Inventory'] - part_data['Quantity Received YTD']) + part_data['COGS'],
                                                (part_data['Ending Inventory'] - part_data['Quantity Ordered YTD']) + part_data['COGS'])

    part_data['Starting Inventory'] = (part_data['Ending Inventory'] - part_data['Quantity Ordered YTD']) + part_data['COGS']
    part_data['Average Inventory'] = (part_data['Starting Inventory'] + part_data['Ending Inventory']) / 2
   
    part_data['Turnover'] = np.where((part_data['COGS'] == 0) & (part_data['Quantity'] != 0),
                                    part_data['Sales - This Year'],
                                    part_data['COGS'] / part_data['Average Inventory'])

    part_data['Turnover'] = part_data.loc[part_data['COGS'] >= 0, 'Turnover'] = (part_data['COGS'] / part_data['Average Inventory'])
    part_data.loc[part_data['Average Inventory'] == 0, 'Turnover'] = 0
    
   
    
    result_df = part_data.groupby(dataframe.columns.tolist()).agg({
        'COGS': 'sum',
        'Starting Inventory': 'sum',
        'Ending Inventory': 'sum',
        'Average Inventory': 'sum',
        'Turnover': 'mean'
    }).reset_index()

    #result_df['Turnover'] = np.where(result_df['Turnover'] =)
    
    return result_df

complete_parts_data = create_part_dataframe(parts_data)

In [1579]:
def convert_type(dataframe, column_names, float_columns):
    for col in column_names:
        if col in float_columns:
            dataframe[col] = dataframe[col].astype('float64')
        elif dataframe[col].dtype != 'object':
            if (all(dataframe[col] % 1 == 0)) and (all(dataframe[col] >= -127) and all(dataframe[col] <= 127)):
                dataframe[col] = dataframe[col].astype('int8')
            else:
                dataframe[col] = dataframe[col].astype('int64')
        else:
            try:
                dataframe[col] = dataframe[col].astype('float64')
            except ValueError:
                pass  # Skip columns that can't be converted to float
    return dataframe


float_columns = ['Margin %','Margin', 'Price', 'Gross Profit -  Last Month',
       'Gross Profit -  Last 3 Months', 'Gross Profit -  Last 6 Months',
       'Gross Profit -  Last 9 Months', 'Gross Profit -  Last 12 Months',
       'Gross Profit -  Last 2 Years', 'Gross Profit -  Last 3 Years',
       'Gross Profit -  Last 4 Years', 'Gross Profit -  Last 5 Years',
       'Gross Profit -  Last 10 Years', 'Total Gross Profit',
       'Gross Profit - Jan', 'Gross Profit - Feb', 'Gross Profit - Mar',
       'Gross Profit - Apr', 'Gross Profit - May', 'Gross Profit - Jun',
       'Gross Profit - Jul', 'Gross Profit - Aug', 'Gross Profit - Sep',
       'Gross Profit - Oct', 'Gross Profit - Nov', 'Gross Profit - Dec',
       'Turnover', 'COGS', 'Cost Per Unit', 'Total Cost']

columns = ['Quantity', 'Sales Last Month',
       'Sales Last 3 Months', 'Sales Last 6 Months', 'Sales Last 9 Months',
       'Sales Last 12 Months', 'Sales Last 2 Years', 'Sales Last 3 Years',
       'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years',
       'Months No Sale', 'Reorder Point', 'Sales - Jan', 'Sales - Feb',
       'Sales - Mar', 'Sales - Apr', 'Sales - May', 'Sales - Jun',
       'Sales - Jul', 'Sales - Aug', 'Sales - Sep', 'Sales - Oct',
       'Sales - Nov', 'Sales - Dec', 'Sales - 1st Qtr', 'Sales - 2nd Qtr',
       'Sales - 3rd Qtr', 'Sales - 4th Qtr', 'Sales - This Year',
       'Sales - Last Year', 'Quantity Ordered YTD', 'Obsolete']
complete_parts_data = convert_type(complete_parts_data, columns, float_columns)

In [1583]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

time_period = ['Sales - Jan', 'Sales - Feb',
       'Sales - Mar', 'Sales - Apr', 'Sales - May', 'Sales - Jun',
       'Sales - Jul', 'Sales - Aug', 'Sales - Sep', 'Sales - Oct',
       'Sales - Nov', 'Sales - Dec', 'Sales - 1st Qtr', 'Sales - 2nd Qtr',
       'Sales - 3rd Qtr', 'Sales - 4th Qtr', 'Sales - Last Year']

def find_best_alpha(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    alpha_values = np.linspace(0.01, 0.99, num=20)

    best_alpha = None
    best_mse = float('inf')

    for alpha in alpha_values:
        model = ExponentialSmoothing(y_train, seasonal='add', seasonal_periods=12, trend='add')
        fit_model = model.fit(smoothing_level=alpha)
        forecast = fit_model.forecast(steps=len(X_test))

        mse = mean_squared_error(y_test, forecast)
        if mse < best_mse:
            best_mse = mse
            best_alpha = alpha

    return best_alpha, best_mse

# Replace 'time_period' with the actual column name for your X data
time_period = 'Sales - Last Year'  # Change this to your desired column
X = parts_data[time_period]
y = parts_data['Sales - This Year']

best_alpha, best_mse = find_best_alpha(X, y)
print(f"Best Alpha: {best_alpha}, Best MSE: {best_mse}")

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(da

Best Alpha: 0.99, Best MSE: 141.26608748531575


In [1587]:
complete_parts_data.head()

,Part Number,Category,Description,Supplier Name,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,...,Gross Profit - Aug,Gross Profit - Sep,Gross Profit - Oct,Gross Profit - Nov,Gross Profit - Dec,COGS,Starting Inventory,Ending Inventory,Average Inventory,Turnover
0,0,PM1,return spring,triumph,100.0,7.26,7.26,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0 580 463 999,ACC,euro moto internationalrelectrics bosch,euro moto internationalrelectrics,100.0,629.99,629.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00-0450,ACC,"screw, gearbox,inspec&",moto international,100.0,1.95,1.95,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00-548588,ATA,tube 150/90- 15 centre90' stem,butler maps inc,100.0,19.99,19.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00-F01740401,ACC,turn signal light control,moto international,100.0,166.99,166.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Example usage
average_inventory_level = parts_data['Average Inventory'].abs()
carrying_cost_rate = 0.125
depreciation = 0.1
avg_time_in_inventory = parts_data['Months No Sale']

In [1600]:
import pandas as pd
import numpy as np
import math
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Define functions
def calculate_demand(sales_data, alpha):
    initial_forecast = sales_data * 1.15
    forecast = alpha * sales_data + (1 - alpha) * initial_forecast
    return forecast

def calculate_ordering_cost(order_quantity, fixed_ordering_costs, carrying_cost_rate, avg_lead_time_days, sales_data):
    lead_time_demand = (sales_data / len(sales_data)) * avg_lead_time_days
    carrying_costs_lead_time = 0.5 * carrying_cost_rate * lead_time_demand
    total_ordering_costs = fixed_ordering_costs + carrying_costs_lead_time * order_quantity
    return total_ordering_costs

def calculate_carrying_cost(average_inventory_level, carrying_cost_rate, avg_time_in_inventory, depreciation):
    carrying_cost = (
        average_inventory_level * carrying_cost_rate * avg_time_in_inventory +
        average_inventory_level * depreciation
    )
    return carrying_cost

def calculate_eoq(row):
    if row['Carrying Cost'] > 0:
        return math.sqrt((2 * row['Demand'] * row['Ordering Cost']) / row['Carrying Cost'])
    else:
        return row['Demand'] * math.sqrt(2 * row['Ordering Cost'])

complete_parts_data['Demand'] = calculate_demand(complete_parts_data['Sales - This Year'], alpha=0.2163157894736842)
complete_parts_data['Ordering Cost'] = calculate_ordering_costs(
    order_quantity=parts_data['Quantity Ordered YTD'].sum(),
    fixed_ordering_costs=parts_data['Total Cost'] * 0.3,
    carrying_cost_rate=0.20,
    avg_lead_time_days=14,
    sales_data=parts_data['Sales - This Year']
)
complete_parts_data['Carrying Cost'] = calculate_carrying_cost(
    average_inventory_level=complete_parts_data['Average Inventory'].abs(),
    carrying_cost_rate=0.125,
    avg_time_in_inventory=parts_data['Months No Sale'],
    depreciation=0.1
)
complete_parts_data['EOQ'] = complete_parts_data.apply(calculate_eoq, axis=1)

In [1601]:
turnover_mask = parts_data_funct['Turnover'] < 0
cols_to_use = ['Part Number', 'Description', 'Supplier Name','Price', 'Quantity',
               'Quantity Ordered YTD','Quantity Received YTD','Cost Per Unit', 'COGS', 
               'Starting Inventory', 'Ending Inventory', 'Average Inventory', 'Sales Last 3 Months', 'Sales Last 6 Months', 'Sales Last 9 Months',
               'Sales - This Year', 'Months No Sale',
               'Turnover', 'Demand', 'EOQ']

negative_turn = parts_data_funct[turnover_mask][cols_to_use]
negative_turn.to_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Dealership_Data/negative_turn.csv')
negative_turn

,Part Number,Description,Supplier Name,Price,Quantity,Quantity Ordered YTD,Quantity Received YTD,Cost Per Unit,COGS,Starting Inventory,Ending Inventory,Average Inventory,Sales Last 3 Months,Sales Last 6 Months,Sales Last 9 Months,Sales - This Year,Months No Sale,Turnover,Demand,EOQ
594,02-13777S,8mmx40 chr buttonhead (10),moto international,17.99,0,30,30,0.90,2.70,-27.30,0.00,-13.650,0,0,3,3,6,-0.197802,3.352658,0.359063
1982,04-148C,bulb 12v-45/45w p45t h4,moto international,0.99,0,1,1,0.15,0.60,-0.40,0.00,-0.200,0,0,4,4,6,-3.000000,4.470211,3.955130
3892,09-88064,shiftlever rubber yam 132-,moto international,1.99,0,15,15,0.20,2.00,-13.00,0.00,-6.500,0,0,10,10,6,-0.307692,11.175526,1.734439
8720,11711671AA,extension,ducati,13.99,0,55,846,10.46,10.46,-44.54,0.00,-22.270,0,0,1,1,6,-0.469690,1.117553,0.093703
16075,2080043-T0301,"rubber,gearchange",triumph,11.99,0,40,40,7.68,7.68,-32.32,0.00,-16.160,1,1,2,1,0,-0.475248,1.117553,0.320704
37010,46531231646,side stand spring,bmw,12.90,0,10,54,3.78,3.78,-6.22,0.00,-3.110,0,0,0,1,0,-1.215434,1.117553,0.731047
39488,46717687107,"thrust washer, left",bmw,6.99,0,6,3334,4.42,4.42,-1.58,0.00,-0.790,1,1,1,1,1,-5.594937,1.117553,0.966987
61878,83300413585,repair connector-3 pole,bmw,25.28,0,113,1739,16.85,16.85,-96.15,0.00,-48.075,0,0,2,1,7,-0.350494,1.117553,0.059547
82790,T2020472,"circlip, m'cylinder, rear",triumph,3.99,0,15,684,1.85,1.85,-13.15,0.00,-6.575,0,0,1,1,6,-0.281369,1.117553,0.172452
84707,T2101015,"cap, expansion tank",triumph,2.99,0,4,4,1.50,1.50,-2.50,0.00,-1.250,1,1,1,1,0,-1.200000,1.117553,1.153109
